In [1]:
import numpy as np
import math
from numpy.linalg import norm

In [10]:
newroads = [
    ((411, 172), (374, 141)),
    ((673, 448), (607, 468)),
    ((441, 458), (443, 509)),
    ((673, 448), (637, 608)),
    ((753, 434), (726, 535)),
    ((623, 676), (607, 740)),
    ((566, 328), (551, 303)),
    ((735, 874), (769, 885)),
    ((673, 762), (665, 800)),
    ((278, 789), (283, 756)),
    ((276, 818), (278, 789)),
    ((597, 778), (607, 740)),
    ((527, 561), (539, 558)),
    ((726, 535), (673, 448)),
    ((596, 671), (568, 676)),
    ((540, 627), (515, 625)),
    ((278, 94), (295, 77)),
    ((475, 367), (505, 360)),
    ((498, 498), (443, 509)),
    ((349, 114), (345, 95)),
    ((673, 448), (607, 314)),
    ((443, 424), (441, 458)),
    ((310, 741), (246, 704)),
    ((645, 752), (649, 728)),
    ((295, 77), (293, 46)),
    ((808, 815), (902, 863)),
    ((345, 95), (342, 81)),
    ((491, 441), (441, 458)),
    ((708, 866), (714, 845)),
    ((80, 825), (140, 823)),
    ((821, 780), (842, 719)),
    ((272, 144), (271, 112)),
    ((71, 332), (84, 333)),
    ((484, 393), (455, 404)),
    ((970, 647), (896, 639)),
    ((553, 483), (498, 498)),
    ((562, 545), (553, 584)),
    ((411, 172), (465, 189)),
    ((750, 783), (808, 815)),
    ((435, 565), (419, 515)),
    ((505, 552), (543, 541)),
    ((569, 652), (562, 629)),
    ((645, 752), (639, 761)),
    ((271, 112), (278, 94)),
    ((802, 615), (854, 647)),
    ((597, 414), (607, 468)),
    ((539, 558), (528, 601)),
    ((90, 303), (71, 319)),
    ((568, 676), (561, 729)),
    ((567, 241), (546, 216)),
    ((401, 193), (406, 180)),
    ((541, 832), (585, 841)),
    ((80, 825), (35, 825)),
    ((429, 817), (494, 827)),
    ((479, 563), (466, 561)),
    ((673, 762), (697, 658)),
    ((808, 815), (821, 780)),
    ((902, 863), (938, 755)),
    ((755, 578), (802, 615)),
    ((465, 189), (501, 200)),
    ((342, 81), (336, 3)),
    ((135, 287), (90, 303)),
    ((726, 535), (704, 637)),
    ((374, 141), (349, 114)),
    ((651, 858), (708, 866)),
    ((561, 729), (518, 705)),
    ((394, 207), (319, 145)),
    ((204, 786), (175, 787)),
    ((543, 541), (550, 528)),
    ((651, 858), (646, 887)),
    ((461, 548), (435, 565)),
    ((952, 704), (973, 655)),
    ((254, 202), (271, 213)),
    ((595, 793), (597, 778)),
    ((607, 314), (567, 241)),
    ((595, 590), (637, 608)),
    ((952, 704), (938, 755)),
    ((498, 498), (505, 552)),
    ((466, 561), (461, 548)),
    ((631, 633), (623, 676)),
    ((568, 676), (569, 652)),
    ((401, 193), (460, 206)),
    ((536, 425), (519, 387)),
    ((575, 477), (562, 545)),
    ((231, 779), (231, 820)),
    ((206, 734), (204, 786)),
    ((561, 729), (517, 757)),
    ((429, 817), (276, 818)),
    ((219, 283), (204, 280)),
    ((842, 719), (938, 755)),
    ((431, 762), (429, 817)),
    ((412, 63), (435, 84)),
    ((270, 164), (272, 144)),
    ((607, 740), (645, 752)),
    ((607, 314), (566, 328)),
    ((84, 333), (119, 322)),
    ((842, 719), (743, 673)),
    ((597, 778), (556, 768)),
    ((460, 206), (502, 213)),
    ((902, 863), (893, 887)),
    ((336, 3), (346, 1)),
    ((494, 827), (484, 877)),
    ((902, 863), (976, 878)),
    ((704, 637), (697, 658)),
    ((585, 841), (651, 858)),
    ((460, 206), (465, 189)),
    ((645, 752), (673, 762)),
    ((607, 468), (575, 477)),
    ((134, 730), (206, 734)),
    ((546, 216), (501, 200)),
    ((276, 818), (231, 820)),
    ((140, 823), (176, 817)),
    ((578, 623), (585, 599)),
    ((71, 319), (71, 332)),
    ((505, 360), (519, 387)),
    ((498, 498), (491, 441)),
    ((651, 858), (665, 800)),
    ((254, 202), (270, 164)),
    ((424, 419), (443, 424)),
    ((562, 629), (540, 627)),
    ((502, 213), (501, 200)),
    ((443, 509), (461, 548)),
    ((524, 288), (487, 273)),
    ((393, 54), (412, 63)),
    ((735, 874), (739, 864)),
    ((490, 621), (515, 625)),
    ((556, 768), (541, 832)),
    ((551, 303), (524, 288)),
    ((536, 425), (553, 483)),
    ((484, 393), (491, 441)),
    ((293, 46), (257, 0)),
    ((697, 658), (631, 633)),
    ((566, 328), (597, 414)),
    ((440, 247), (394, 207)),
    ((435, 565), (460, 596)),
    ((821, 780), (787, 766)),
    ((419, 515), (424, 419)),
    ((585, 599), (605, 600)),
    ((586, 576), (595, 590)),
    ((515, 625), (505, 552)),
    ((231, 820), (229, 887)),
    ((135, 783), (134, 730)),
    ((542, 610), (540, 627)),
    ((35, 825), (20, 826)),
    ((708, 866), (735, 874)),
    ((204, 280), (171, 277)),
    ((489, 763), (431, 762)),
    ((424, 419), (432, 392)),
    ((231, 820), (176, 817)),
    ((494, 827), (541, 832)),
    ((637, 608), (631, 633)),
    ((505, 552), (479, 563)),
    ((517, 757), (489, 763)),
    ((938, 755), (975, 767)),
    ((553, 584), (542, 610)),
    ((578, 623), (564, 614)),
    ((550, 528), (553, 483)),
    ((595, 793), (611, 799)),
    ((429, 817), (425, 885)),
    ((673, 762), (750, 783)),
    ((854, 647), (952, 704)),
    ((564, 614), (565, 597)),
    ((726, 535), (755, 578)),
    ((665, 800), (597, 778)),
    ((394, 207), (401, 193)),
    ((519, 387), (484, 393)),
    ((487, 273), (440, 247)),
    ((769, 885), (808, 815)),
    ((607, 468), (586, 576)),
    ((541, 832), (532, 885)),
    ((432, 392), (475, 367)),
    ((176, 817), (175, 787)),
    ((171, 277), (135, 287)),
    ((575, 477), (553, 483)),
    ((665, 800), (696, 810)),
    ((970, 647), (969, 626)),
    ((567, 241), (579, 231)),
    ((491, 441), (536, 425)),
    ((346, 1), (386, 42)),
    ((585, 841), (595, 793)),
    ((175, 787), (135, 783)),
    ((431, 762), (310, 741)),
    ((460, 596), (490, 621)),
    ((743, 673), (697, 658)),
    ((246, 704), (69, 694)),
    ((607, 740), (561, 729)),
    ((631, 633), (578, 623)),
    ((406, 180), (411, 172)),
    ((973, 655), (970, 647)),
    ((455, 404), (443, 424)),
    ((821, 780), (865, 797)),
    ((216, 779), (231, 779)),
    ((556, 768), (561, 729)),
    ((623, 676), (596, 671))
]

In [3]:
# const
HEIGHT = 1000
WIDTH = 1000

INF = 99999.999


In [4]:

class Point:
    def __init__(self, pos):
        self.pos = pos
        self.x, self.y = pos
        self.adjacents = [] # (road, adj_point)
        
    def _calc_dist(self, pos):
        return math.dist(self.pos, pos)
    
    def __eq__(self, _point: object):
        return self.x == _point.x and self.y == _point.y
    
    def __str__(self) -> str:
        return f'{self.pos}'

In [5]:
class Road:
    def __init__(self, from_point, to_point):
        self.from_point = from_point
        self.to_point = to_point
        self.from_pos = from_point.pos
        self.to_pos = to_point.pos
        self.length = math.dist(from_point.pos, to_point.pos)
        
        self.from_point.adjacents.append( (self, to_point) )
        self.to_point.adjacents.append( (self, from_point) )
    
    def _calc_dist(self, pos):
        p1 = np.asarray(self.from_pos)
        p2 = np.asarray(self.to_pos)
        p3 = np.asarray(pos)
        return np.abs(np.cross(p2-p1, p1-p3)) / norm(p2-p1)
    
    def _is_near(self, pos):
        x,y = pos
        x1,y1 = self.from_pos
        x2,y2 = self.to_pos
        
        is_x_middle = (x >= x1 and x <= x2) or (x <= x1 and x >= x2)
        is_y_middle = (y >= y1 and y <= y2) or (y <= y1 and y >= y2)
        
        return self._calc_dist(pos) <= 20 and (is_x_middle or is_y_middle)
    
    def __str__(self) -> str:
        return f'({self.from_pos}, {self.to_pos})'

In [6]:
class Dragger:
    def __init__(self) -> None:
        self.dragging = False

In [7]:
points = list(set([ x for x, y in roads]))
str_points = [ str(x) for x in points]
map_points = {}
for point in points:
    map_points[str(point)] = Point(point)
    
roads_lst = []
for from_pos, to_pos in roads:
    roads_lst.append(Road(map_points[str(from_pos)], map_points[str(to_pos)]))
    
for key in map_points:
    map_points[key].f = INF
    map_points[key].g = INF
    map_points[key].h = INF
    map_points[key].parent = None


In [8]:
start_pos = (14,131)
end_pos = (886, 558)

#Initialize
map_points[str(start_pos)].f = 0
map_points[str(start_pos)].g = 0
map_points[str(start_pos)].h = 0
map_points[str(start_pos)].parent = None

openlist = {str(start_pos)}
closedlist = set()
route_found = False

while len(openlist) > 0 and not route_found:
    #find the node with the least f on open list -> q
    minf = min([map_points[key].f for key in openlist])
    q = None
    for key in openlist:
        if minf == map_points[key].f:
            q = key
            break
    
    #pop q off open list
    openlist.remove(q)
    closedlist.add(q)
    
    #generate q's successors and set their parent to q
    point = map_points[q]
    for road, to_point in point.adjacents:
        
        #if successor is the goal, stop search
        current_f = to_point.f
        if to_point.pos == end_pos:
            to_point.parent = point
            print("FOUND")
            route_found = True
            break
            #trace path
            
            break
        elif not str(to_point) in closedlist:
            g_new = point.g + road.length
            h_new = point._calc_dist(end_pos)
            f_new = g_new + h_new
            
            if str(to_point) not in openlist or current_f > f_new:
                openlist.add(str(to_point))
                
                to_point.f = f_new
                to_point.g = g_new
                to_point.h = h_new
                to_point.parent = point

FOUND


In [9]:
node = map_points[str(end_pos)]
while node.parent is not None:
    print(node)
    node = node.parent
print(node.pos)

(886, 558)
(767, 540)
(643, 540)
(618, 557)
(598, 534)
(545, 481)
(552, 438)
(524, 394)
(553, 301)
(551, 203)
(482, 220)
(410, 217)
(360, 217)
(332, 193)
(248, 153)
(227, 130)
(115, 145)
(45, 136)
(14, 131)
